In [ ]:
import pulp
import numpy as np

In [ ]:
def transformer_jeu_en_probleme_optimisation(A, B, verbose=True):
    prob = pulp.LpProblem("Optimization Problem", pulp.LpMinimize)

    m = A.shape[0]
    n = A.shape[1]

    # VARIABLES

    xa, xb = [], []
    ra, rb = [], []
    sa, sb = [], []

    def init(i, x, r, s, letter):
        x.append(pulp.LpVariable(f"x{letter}_{i}", lowBound=0, upBound=1)) # [0, 1]
        r.append(pulp.LpVariable(f"r{letter}_{i}", lowBound=0)) # >= 0
        s.append(pulp.LpVariable(f"s{letter}_{i}", cat=pulp.LpBinary)) # {0, 1}

    for i in range(m) :
        init(i, xa,ra,sa, "a")
    for j in range(n):
        init(j, xb,rb,sb, "b")

    gain_a = pulp.LpVariable("gain_a", lowBound=0)
    gain_b = pulp.LpVariable("gain_b", lowBound=0)

    # for i in range(m):
    #     prob += gain_a == pulp.lpSum(xa[i] * A[i, j] * xb[j] for j in range(n))

    # for j in range(n):
    #     prob += gain_b == pulp.lpSum(xb[j] * B[i, j] * xa[i] for i in range(n))

    # CONSTRAINTS


    # ---------------------------

    prob = pulp.lpSum(ra) + pulp.lpSum(rb)

    prob.solve()
    
    if verbose :
        print(f"Status: {pulp.LpStatus[prob.status]}")
        for v in prob.variables():
            print(f"{v.name}: {v.value()}")
        print(f"Objective: {pulp.value(prob.objective)}")
    
    solution = pulp.value(prob.objective)

    return solution

In [ ]:
def tester_equilibre_de_nash(matrice_A, matrice_B, equilibre_attendu):
    solution = transformer_jeu_en_probleme_optimisation(matrice_A, matrice_B)
    assert solution == equilibre_attendu, f"Erreur: l'équilibre de Nash n'est pas correct. Attendu: {equilibre_attendu}, Obtenu: {solution}"

In [ ]:
tester_equilibre_de_nash(np.array([[3, 2], [1, 4]]), np.array([[2, 1], [3, 2]]), 3)
tester_equilibre_de_nash(np.array([[1, 2], [3, 4]]), np.array([[4, 3], [2, 1]]), 2)

print("Tous les tests de l'équilibre de Nash ont réussi.")